In [1]:
from langchain.text_splitter import MarkdownTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document

from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

1. 데이터 로드
2. 텍스트 분할
3. 인덱싱(임베딩 후 인덱싱)
4. 검색(retrieval)

In [2]:
import torch

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.backends.mps.is_available() else "cpu")
device

device(type='cpu')

In [3]:
input_pdf = "datasets/manual.pdf"

# input_pdf = "datasets/여비산정기준표.pdf"
preprocessed_file = "datasets/ocr_output.pdf"
result_file = "datasets/ocr_output_result.pdf",

In [4]:
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode

pipeline_options = PdfPipelineOptions(
    do_table_structure = True,
    do_text_extraction=True,
)
pipeline_options.table_structure_options.mode = TableFormerMode.ACCURATE

doc_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

doc = doc_converter.convert(input_pdf)

markdown_text = doc.document.export_to_markdown()
print(markdown_text)
#읽지 못하는 텍스트와 사진 처리방법 고민

## Ⅴ   사업비 집행 및 정산지침

## 1 사업비 지급 및 집행기간

<!-- image -->

## □ 사업비 지급

| 1~2분기      | 1~2분기   | 1~2분기                 | 1~2분기   | 3~4분기                 | 3~4분기   | 3~4분기      | 3~4분기   |
|--------------|-----------|-------------------------|-----------|-------------------------|-----------|--------------|-----------|
| 1회차        | 1회차     | 2회차                   | 2회차     | 3회차                   | 3회차     | 4회차        | 4회차     |
| 신청 시기    | 지급 비율 | 신청 시기               | 지급 비율 | 신청 시기               | 지급 비율 | 신청 시기    | 지급 비율 |
| 협약 체결 시 | 20%       | 협약 체결 후 2개월 이내 | 46%       | 중간 평가 후 1개월 이내 | 9%        | 4분기 (10월) | 25%       |

※ 지급비율은 변동될 수 있으며, 중간평가 결과에 따라 3회차부터 지급을 유예할 수 있음

## □ 사업비 집행기간

- ○ '24. 1. 1. ~ '24. 12. 31. ※  동  기간  이외의 사용금액은 불인정 금액으로 간주하고 전액 반납해야 함 ※  단,  집행기간  연장에 대해서 사전 승인을 받을 시에는 그 기한까지는 인정

## 2 사업비 계좌관리 및 사용원칙

## □ 사업비 계좌관리

- ○ ( 별도 계좌개설 필수 ) 각 지원 대상자는 당해 랩 운영지원금과 민간부담금을 관리하는 통장법인명의을 주관 참여기관 별로 각각 ( ) · 개설하고 해당 계좌와 연계된 연구비카드 , (KCA) 만 개설하여 사용

## □ 사업비 사용원

In [5]:
import pdfplumber
import pandas as pd

def table_chunker(pdf_path: str):
    chunks = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                # table: 2D 리스트 (헤더 포함)
                try:
                    df = pd.DataFrame(table[1:], columns=table[0])
                except:
                    df = pd.DataFrame(table)
                md_table = df.to_markdown(index=False)
                chunks.append({
                    "text": md_table,
                    "metadata": {
                        "type": "table",
                        "page": page.page_number
                    }
                })
    return chunks

# 사용 예시
table_chunks = table_chunker(input_pdf)

In [6]:
from docling.chunking import HybridChunker

chunker = HybridChunker(
    tokenizer='BAAI/bge-m3',
    max_token_length=512,
    overlap=100
)

chunks = list(chunker.chunk(doc.document))

# chunks는 청커로 생성된 문서 조각들
# documents = [Document(page_content=chunk.text, metadata={"source": chunk.meta}) for chunk in chunks]

In [7]:
text_documents = []
for chunk in chunks:
    text = chunk.text

    source = input_pdf

    # page_no를 추출 (여러 Prov 아이템 중 첫 번째의 page_no를 사용하거나, set으로 처리)
    page_nos = sorted(
        set(
            prov.page_no
            for item in chunk.meta.doc_items
            for prov in item.prov
            if hasattr(prov, "page_no")
        )
    ) if hasattr(chunk.meta, "doc_items") else []

    # 가장 근접한 헤딩(섹션 이름) 추출 (예: 마지막 요소)
    headings = chunk.meta.headings if hasattr(chunk.meta, "headings") else []
    section = headings[-1] if headings else None

    metadata = {
        "source": source,
        "page_numbers": page_nos,
        "section": section
    }
    # (3) LangChain Document 생성
    text_documents.append(Document(page_content=text, metadata=metadata))

In [8]:
# table_chunks를 Document 형식으로 변환
table_documents = [
    Document(
        page_content=chunk["text"],
        metadata={
            "type": "table",
            "page": chunk["metadata"]["page"],
            "source": input_pdf
        }
    ) for chunk in table_chunks
]

# 두 Document 리스트 합치기
all_documents = table_documents + text_documents

In [9]:
embedding_model = HuggingFaceEmbeddings(
    model_name="dragonkue/BGE-m3-ko",
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True},
)

vectorstore = FAISS.from_documents(
    documents=all_documents,
    embedding=embedding_model
)

retriever = vectorstore.as_retriever()

/var/folders/f1/c1mlzgw562581ych5vk1w4tr0000gn/T/ipykernel_46901/3791966284.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [10]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# 모델 초기화
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

compressor = CrossEncoderReranker(model=model, top_n=8)

# 문서 압축 검색기 초기화
compression_reranker = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [12]:
from langchain_ollama import OllamaLLM

# llm = OllamaLLM(model="llama3.2:3b")
# llm = OllamaLLM(model="deepseek-r1:1.5b")
llm = OllamaLLM(model="gemma3:4b")
# llm = ChatOpenAI(model_name = "gpt-4o-mini", temperature=0)

def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

prompt = ChatPromptTemplate.from_template("""
You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question.
If the user asks for a simple answer, summarize the key points.
If the question is unrelated to the context in the regulations, respond with "관련 정보를 찾을 수 없습니다."
You must answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:
""")

rag_chain = (
    {'context': compression_reranker | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
question = "교수가 미국으로 4일 출장을 다녀오면 받을 수 있는 숙박비는?"
result = rag_chain.invoke(question)
print(result)

교수가 미국으로 4일 출장을 다녀오면 숙박비는 $35(1일당) x 4일 = $140를 선지급받을 수 있습니다.


In [ ]:
# 출처 기능까지 지원하는지?

# restapi